## Lesson 7: Build a Crew to Tailor Job Applications

https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/16/build-a-crew-to-trailor-job-applications-(code)

In this lesson you will build your first multi-agent system.

In [1]:
import warnings

from crewai import Agent, Crew, Task
from crewai_tools import FileReadTool, MDXSearchTool, ScrapeWebsiteTool, SerperDevTool
from dotenv import find_dotenv, load_dotenv
from IPython.display import Markdown, display

warnings.filterwarnings("ignore")

In [2]:
load_dotenv(find_dotenv())

True

### crewAI Tools

In [3]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path="./fake_resume.md")
semantic_search_resume = MDXSearchTool(mdx="./fake_resume.md")

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it]


### Creating Agents

In [4]:
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on job posting to help job applicants.",
    backstory=(
        "As a Job Researcher, your prowess in navigating and extracting critical "
        "information from job postings is unmatched. Your skills help pinpoint the necessary "
        "qualifications and skills sought by employers, forming the foundation for "
        "effective application tailoring."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
)

In [5]:
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants to help them stand out in the job market.",
    backstory=(
        "Equipped with analytical prowess, you dissect and synthesize information "
        "from diverse sources to craft comprehensive personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
)

In [6]:
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a resume stand out in the job market.",
    backstory=(
        "With a strategic mind and an eye for detail, you excel at refining resumes "
        "to highlight the most relevant skills and experiences, ensuring "
        "they resonate perfectly with the job's requirements."
    ),
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
)

In [7]:
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points based on "
    "the resume and job requirements.",
    backstory=(
        "Your role is crucial in anticipating the dynamics of the interviews. "
        "With your ability to formulate key questions and talking points, "
        "you prepare candidates for success, ensuring they can confidently "
        "address all aspects of the job they are applying for."
    ),
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
)

### Creating Tasks

In [8]:
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True,
)

In [9]:
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True,
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run unitil it has the output(s) from those tasks.

In [10]:
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist,
)

In [11]:
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer,
)

### Creating the Crew

In [12]:
job_application_crew = Crew(
    agents=[researcher, profiler, resume_strategist, interview_preparer],
    tasks=[
        research_task,
        profile_task,
        resume_strategy_task,
        interview_preparation_task,
    ],
    verbose=True,
)

### Runnign the Crew

In [13]:
job_application_inputs = {
    "job_posting_url": "https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund",
    "github_url": "https://github.com/joaomdmoura",
    "personal_writeup": """
        Noah is an accomplished Software Engineering Leader with 18
        years of experience, specializing in managing remote and
        in-office teams, and expert in multiple programming languages
        and frameworks. He holds an MBA and a strong background in AI
        and data science. Noah has successfully led major tech initiatives
        and startups, proving his ability to drive innovation and growth
        in the tech industry. Ideal for leadership roles that require
        a strategic and innovative approach.
    """,
}

In [14]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (
        Noah is an accomplished Software Engineering Leader with 18
        years of experience, specializing in managing remote and
        in-office teams, and expert in multiple programming languages
        and frameworks. He holds an MBA and a strong background in AI
        and data science. Noah has successfully led major tech initiatives
        an

In [15]:
display(Markdown(data="./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, specializing in managing both remote and in-office engineering teams. His expertise includes software development, process innovation, and team collaboration. Proficient in Ruby, Python, JavaScript, TypeScript, and Elixir, with deep knowledge in various front-end frameworks. Experienced in data science and machine learning, leading successful AI deployments and innovative data model development.

## Work Experience

### DataKernel: Director of Software Engineering (remote) - 2022-Present
- Transformed engineering into a revenue pillar, expanding the customer base and integrating AI technologies and scalable databases.
- Led skill development and strategic project goals, setting benchmarks in AI technology adoption.

### DataKernel: Senior Software Engineering Manager (remote) - 2019-2022
- Directed engineering strategy and operations, managed global teams, and recruited top talent for continuous improvement.

### InnovPet: Founder & CEO (remote) - 2019-2022
- Founded startup focused on innovative IoT solutions for pet care, overseeing product development and establishing market presence.

### EliteDevs: Engineering Manager (remote) - 2018-2019
- Formulated strategic plans for inter-departmental coordination, managed engineering teams, and implemented goal-setting frameworks.

### PrintPack: Engineering Manager (remote) - 2016-2018
- Led high-performance team, integrated data analytics, and developed predictive modeling tool for customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) - 2015-2016
- Optimized central API, enhanced system performance, and implemented caching strategies for improved user experience.

### BetCraft: CTO - 2013-2015
- Led technology strategy post-Series A funding, improved platform performance, and expanded market reach.

## Education

- MBA in Information Technology - London Business School
- Advanced Leadership Techniques Certification - University of London
- Data Science Specialization Certification - Coursera (Johns Hopkins University)
- B.Sc. in Computer Science - University of Edinburgh

Noah Williams excels in senior executive roles, offering a unique blend of technical expertise and strategic leadership for innovative tech solutions.

In [16]:
display(Markdown(data="./interview_materials.md"))

Based on Noah Williams' extensive experience in software engineering leadership, remote team management, and innovative tech solutions, here are some potential interview questions and talking points tailored to his background and the job requirements:

1. Can you elaborate on your experience in transforming engineering teams into revenue pillars and expanding customer bases through innovative technologies, as seen in your role at DataKernel?
2. How do you approach strategic project goals and set benchmarks in AI technology adoption, as demonstrated in your tenure at DataKernel?
3. What methodologies do you employ to recruit and develop top talent in global teams while ensuring continuous improvement, as shown in your role as a Senior Software Engineering Manager at DataKernel?
4. How did your entrepreneurial journey with InnovPet shape your leadership style and strategic vision for innovative IoT solutions in the pet care industry?
5. In what ways have you leveraged data analytics and predictive modeling tools to drive strategic decision-making and enhance customer behavior analysis, as evidenced in your role at PrintPack?
6. Can you discuss your experience in optimizing central APIs and implementing caching strategies to improve system performance and user experience, as showcased during your time at DriveAI?
7. How have your strategic technology initiatives post-Series A funding at BetCraft contributed to platform performance improvements and market expansion?
8. With your educational background in MBA, Data Science Specialization, and Computer Science, how do you integrate your academic knowledge into practical applications within the tech industry?
9. How do you stay updated on the latest tech trends, especially in AI, machine learning, and data engineering, to drive continuous innovation and growth within organizations?

These questions and discussion points aim to highlight Noah Williams' expertise, leadership skills, and strategic mindset, aligning them with the job requirements for a Full Stack Developer role.